# 05 - Encapsulation in Python (Đóng gói)

> Phase 3: Object-Oriented Programming & Modules

---

## Mục tiêu học tập

Sau khi hoàn thành bài học này, bạn sẽ:

- ✅ Hiểu rõ khái niệm **Encapsulation** (Đóng gói) trong OOP
- ✅ Nắm vững **Access Modifiers**: Public, Protected, Private
- ✅ Thành thạo **@property decorator** cho getters/setters
- ✅ Hiểu **Name Mangling** và cách Python implement private
- ✅ Áp dụng **Data Validation** trong setters
- ✅ Xây dựng **Computed Properties** (read-only properties)
- ✅ Thiết kế classes với encapsulation tốt

---

## 📚 Mục lục

1. [Encapsulation là gì?](#1)
2. [Access Modifiers trong Python](#2)
3. [Name Mangling](#3)
4. [Getters và Setters truyền thống](#4)
5. [@property Decorator](#5)
6. [Computed Properties](#6)
7. [Property Deleter](#7)
8. [Data Validation với Properties](#8)
9. [Read-Only và Write-Only Properties](#9)
10. [Ví dụ thực tế](#10)
11. [Best Practices](#11)
12. [Common Mistakes](#12)
13. [Bài tập thực hành](#13)

---

<a id='1'></a>
## 1. Encapsulation là gì?

### 1.1 Định nghĩa

**Encapsulation** (Đóng gói) là một trong bốn trụ cột của OOP, bao gồm:

1. **Bundling** (Đóng gói): Gói data (attributes) và code (methods) vào trong một đơn vị (class)
2. **Information Hiding** (Ẩn thông tin): Ẩn giấu implementation details, chỉ expose interface cần thiết
3. **Access Control** (Kiểm soát truy cập): Kiểm soát cách data được đọc/ghi

```
┌─────────────────────────────────────────────────────────────────┐
│                      ENCAPSULATION                              │
│                                                                 │
│   ┌─────────────────────────────────────────────────────────┐  │
│   │                     CLASS                                │  │
│   │                                                         │  │
│   │   ┌─────────────────┐    ┌─────────────────────────┐   │  │
│   │   │  PRIVATE DATA   │◄───│  PUBLIC METHODS          │   │  │
│   │   │                 │    │  (Interface)             │   │  │
│   │   │  - __balance    │    │  + deposit()             │   │  │
│   │   │  - __password   │    │  + withdraw()            │   │  │
│   │   │  - _internal    │    │  + get_balance()         │   │  │
│   │   └─────────────────┘    └─────────────────────────┘   │  │
│   │          ▲                          ▲                   │  │
│   │          │ HIDDEN                   │ EXPOSED           │  │
│   └──────────┼──────────────────────────┼───────────────────┘  │
│              │                          │                      │
│              ✗                          ✓                      │
│         Cannot access              Can access                  │
│         directly                   through interface           │
└─────────────────────────────────────────────────────────────────┘
```

### 1.2 Tại sao cần Encapsulation?

| Lợi ích | Mô tả | Ví dụ |
|---------|-------|-------|
| **Data Protection** | Bảo vệ data khỏi bị thay đổi không hợp lệ | Balance không thể âm |
| **Flexibility** | Thay đổi implementation mà không ảnh hưởng code bên ngoài | Đổi cách lưu password |
| **Control** | Kiểm soát cách data được đọc/ghi | Validate trước khi ghi |
| **Simplicity** | Ẩn complexity, expose simple interface | User chỉ cần gọi `deposit()` |
| **Maintainability** | Dễ debug và bảo trì | Bug chỉ có thể ở trong class |

In [ ]:
# Ví dụ: Tại sao cần Encapsulation?

# ❌ KHÔNG có Encapsulation - Data exposed trực tiếp
class BankAccountBad:
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance  # Public - ai cũng có thể thay đổi!

account_bad = BankAccountBad("John", 1000)

# Vấn đề: Có thể set balance thành bất kỳ giá trị nào!
account_bad.balance = -5000  # Không có validation!
account_bad.balance = "invalid"  # Có thể set sai type!
print(f"Bad account balance: {account_bad.balance}")  # "invalid"

print()

In [ ]:
# ✅ CÓ Encapsulation - Data được bảo vệ

class BankAccountGood:
    def __init__(self, owner, balance):
        self._owner = owner
        self.__balance = 0  # Private
        self.deposit(balance)  # Validate through method
    
    @property
    def balance(self):
        """Read-only access to balance."""
        return self.__balance
    
    def deposit(self, amount):
        """Deposit with validation."""
        if not isinstance(amount, (int, float)):
            raise TypeError("Amount must be a number")
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.__balance += amount
        return self.__balance
    
    def withdraw(self, amount):
        """Withdraw with validation."""
        if not isinstance(amount, (int, float)):
            raise TypeError("Amount must be a number")
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if amount > self.__balance:
            raise ValueError("Insufficient funds")
        self.__balance -= amount
        return self.__balance

# Test
account_good = BankAccountGood("John", 1000)
print(f"Good account balance: ${account_good.balance}")

# Deposit works
account_good.deposit(500)
print(f"After deposit: ${account_good.balance}")

# Invalid operations are blocked
try:
    account_good.deposit(-100)  # Blocked!
except ValueError as e:
    print(f"Error: {e}")

try:
    account_good.withdraw(10000)  # Blocked!
except ValueError as e:
    print(f"Error: {e}")

---
<a id='2'></a>
## 2. Access Modifiers trong Python

### 2.1 So sánh với các ngôn ngữ khác

Python **không có strict access modifiers** như Java/C++, mà sử dụng **naming conventions**:

```
┌─────────────────────────────────────────────────────────────────┐
│              ACCESS MODIFIERS COMPARISON                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   JAVA/C++                      PYTHON                          │
│   ─────────                     ──────                          │
│                                                                 │
│   public     ──────────────►    name        (no prefix)         │
│   protected  ──────────────►    _name       (single underscore) │
│   private    ──────────────►    __name      (double underscore) │
│                                                                 │
│   Enforced by compiler          Convention + Name Mangling      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 2.2 Access Levels

| Prefix | Convention | Access | Mô tả |
|--------|------------|--------|-------|
| `name` | Public | Everywhere | Có thể access từ mọi nơi |
| `_name` | Protected | Class & Subclasses | **Convention** - không nên access trực tiếp |
| `__name` | Private | Class only | **Name Mangling** - khó access từ bên ngoài |
| `__name__` | Magic/Dunder | Special | Python internal methods |

In [ ]:
# Access Modifiers Demo

class Employee:
    """
    Employee class demonstrating access modifiers.
    """
    
    # Class attribute (public)
    company = "TechCorp"
    
    def __init__(self, name, salary, ssn):
        # Public - Accessible everywhere
        self.name = name
        
        # Protected - Convention: don't access directly
        self._salary = salary
        
        # Private - Name mangling applied
        self.__ssn = ssn
    
    def get_info(self):
        """Public method - accessible everywhere."""
        return f"{self.name} at {self.company}"
    
    def _calculate_tax(self):
        """Protected method - internal use."""
        return self._salary * 0.2
    
    def __format_ssn(self):
        """Private method - strictly internal."""
        return f"***-**-{self.__ssn[-4:]}"
    
    def display(self):
        """Public method using private/protected internally."""
        tax = self._calculate_tax()
        ssn_masked = self.__format_ssn()
        return f"{self.name}, Salary: ${self._salary:,}, Tax: ${tax:,}, SSN: {ssn_masked}"

# Test
emp = Employee("John Doe", 75000, "123-45-6789")

print("Access Modifiers Demo:")
print("=" * 60)

# PUBLIC - Accessible
print(f"\n1. PUBLIC access:")
print(f"   emp.name = {emp.name}")  # ✓ OK
print(f"   emp.company = {emp.company}")  # ✓ OK
print(f"   emp.get_info() = {emp.get_info()}")  # ✓ OK

# PROTECTED - Accessible but not recommended
print(f"\n2. PROTECTED access (not recommended):")
print(f"   emp._salary = {emp._salary}")  # ⚠ Works but shouldn't
print(f"   emp._calculate_tax() = {emp._calculate_tax()}")  # ⚠ Works but shouldn't

# PRIVATE - Not directly accessible
print(f"\n3. PRIVATE access:")
try:
    print(emp.__ssn)  # ✗ AttributeError
except AttributeError as e:
    print(f"   emp.__ssn -> Error: {e}")

try:
    print(emp.__format_ssn())  # ✗ AttributeError
except AttributeError as e:
    print(f"   emp.__format_ssn() -> Error: {e}")

# Through public method - OK
print(f"\n4. Through PUBLIC interface:")
print(f"   emp.display() = {emp.display()}")

---
<a id='3'></a>
## 3. Name Mangling

### 3.1 Cơ chế hoạt động

Khi bạn dùng `__name` (double underscore prefix), Python **mangles** (xáo trộn) tên thành `_ClassName__name`.

```
┌─────────────────────────────────────────────────────────────────┐
│                     NAME MANGLING                               │
│                                                                 │
│   class MyClass:                                                │
│       def __init__(self):                                       │
│           self.__secret = "hidden"                              │
│                    │                                            │
│                    │ Python transforms to:                      │
│                    ▼                                            │
│           self._MyClass__secret = "hidden"                      │
│                                                                 │
│   obj = MyClass()                                               │
│   obj.__secret         ──► AttributeError ❌                    │
│   obj._MyClass__secret ──► "hidden" ✓ (but don't do this!)     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 3.2 Mục đích của Name Mangling

1. **Tránh name collision** trong inheritance
2. **Signal intent** - chỉ ra đây là internal implementation
3. **Discourage** (không ngăn chặn hoàn toàn) direct access

In [ ]:
# Name Mangling Demo

class Secret:
    def __init__(self):
        self.public = "I'm public"
        self._protected = "I'm protected"
        self.__private = "I'm private"

obj = Secret()

# Xem tất cả attributes
print("All attributes:")
print("=" * 50)
for attr in dir(obj):
    if not attr.startswith('__') or attr.startswith('_Secret'):
        if not attr.startswith('__'):
            print(f"  {attr}")

# Xem chi tiết
print("\nAttribute details:")
print(f"  obj.public = {obj.public}")
print(f"  obj._protected = {obj._protected}")

# __private bị mangle thành _Secret__private
print(f"  obj._Secret__private = {obj._Secret__private}")  # Works but bad practice!

In [ ]:
# Name Mangling trong Inheritance - Tránh name collision

class Parent:
    def __init__(self):
        self.__value = "Parent's private"  # Mangled to _Parent__value
    
    def get_parent_value(self):
        return self.__value

class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__value = "Child's private"  # Mangled to _Child__value (DIFFERENT!)
    
    def get_child_value(self):
        return self.__value

# Test
child = Child()

print("Name Mangling in Inheritance:")
print("=" * 50)

# Không có name collision!
print(f"Parent's __value: {child.get_parent_value()}")
print(f"Child's __value: {child.get_child_value()}")

# Xem các mangled names
print(f"\nMangled attribute names:")
print(f"  _Parent__value = {child._Parent__value}")
print(f"  _Child__value = {child._Child__value}")

---
<a id='4'></a>
## 4. Getters và Setters truyền thống

### 4.1 Java-style Getters/Setters

Trước khi có `@property`, Python developers thường dùng explicit get/set methods:

In [ ]:
# Traditional Getters/Setters (Java-style)

class Rectangle:
    """Rectangle với traditional getters/setters."""
    
    def __init__(self, width, height):
        self.__width = None
        self.__height = None
        self.set_width(width)   # Use setter for validation
        self.set_height(height)
    
    # === Getters ===
    def get_width(self):
        """Get width."""
        return self.__width
    
    def get_height(self):
        """Get height."""
        return self.__height
    
    def get_area(self):
        """Calculated getter."""
        return self.__width * self.__height
    
    def get_perimeter(self):
        """Calculated getter."""
        return 2 * (self.__width + self.__height)
    
    # === Setters ===
    def set_width(self, value):
        """Set width with validation."""
        if not isinstance(value, (int, float)):
            raise TypeError("Width must be a number")
        if value <= 0:
            raise ValueError("Width must be positive")
        self.__width = value
    
    def set_height(self, value):
        """Set height with validation."""
        if not isinstance(value, (int, float)):
            raise TypeError("Height must be a number")
        if value <= 0:
            raise ValueError("Height must be positive")
        self.__height = value

# Usage - Verbose!
rect = Rectangle(5, 3)

print("Traditional Getters/Setters:")
print("=" * 50)

print(f"Width: {rect.get_width()}")
print(f"Height: {rect.get_height()}")
print(f"Area: {rect.get_area()}")
print(f"Perimeter: {rect.get_perimeter()}")

rect.set_width(10)
print(f"\nAfter set_width(10):")
print(f"Width: {rect.get_width()}")
print(f"Area: {rect.get_area()}")

# Validation works
try:
    rect.set_width(-5)
except ValueError as e:
    print(f"\nError: {e}")

### 4.2 Vấn đề với Traditional Getters/Setters

| Vấn đề | Mô tả |
|--------|-------|
| **Verbose** | `obj.get_value()` thay vì `obj.value` |
| **Không Pythonic** | Python khuyến khích attribute access |
| **Breaking change** | Nếu sau này cần thêm validation, phải đổi interface |

---
<a id='5'></a>
## 5. @property Decorator

### 5.1 Giới thiệu

`@property` decorator cho phép:
- Access attributes như thông thường: `obj.value`
- Nhưng vẫn có **control** qua getter/setter

```
┌─────────────────────────────────────────────────────────────────┐
│                    @property DECORATOR                          │
│                                                                 │
│   class Circle:                                                 │
│       @property              ──►  GETTER                        │
│       def radius(self):           obj.radius                    │
│           return self._radius                                   │
│                                                                 │
│       @radius.setter         ──►  SETTER                        │
│       def radius(self, value):    obj.radius = value            │
│           self._radius = value                                  │
│                                                                 │
│       @radius.deleter        ──►  DELETER                       │
│       def radius(self):           del obj.radius                │
│           del self._radius                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 5.2 Cú pháp cơ bản

In [ ]:
# @property Decorator - Basic Syntax

class Circle:
    """Circle với @property decorator."""
    
    def __init__(self, radius):
        self._radius = radius  # Use underscore for internal storage
    
    @property
    def radius(self):
        """
        Getter for radius.
        
        Usage: value = circle.radius
        """
        print("  [Getter called]")
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """
        Setter for radius with validation.
        
        Usage: circle.radius = value
        """
        print(f"  [Setter called with {value}]")
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value

# Usage - Clean syntax!
print("@property Demo:")
print("=" * 50)

circle = Circle(5)

# Access like attribute (but getter is called)
print(f"\nGetting radius:")
r = circle.radius
print(f"  Result: {r}")

# Set like attribute (but setter is called)
print(f"\nSetting radius = 10:")
circle.radius = 10
print(f"  New radius: {circle.radius}")

# Validation works
print(f"\nSetting radius = -5:")
try:
    circle.radius = -5
except ValueError as e:
    print(f"  Error: {e}")

### 5.3 So sánh Traditional vs @property

| Aspect | Traditional | @property |
|--------|------------|----------|
| **Get** | `obj.get_value()` | `obj.value` |
| **Set** | `obj.set_value(x)` | `obj.value = x` |
| **Pythonic** | ❌ No | ✅ Yes |
| **Backward Compatible** | ❌ No | ✅ Yes |

In [ ]:
# Rectangle với @property - Clean version

import math

class Rectangle:
    """Rectangle với @property."""
    
    def __init__(self, width, height):
        self._width = 0
        self._height = 0
        self.width = width    # Use property setter
        self.height = height
    
    # === Width Property ===
    @property
    def width(self):
        """Get width."""
        return self._width
    
    @width.setter
    def width(self, value):
        """Set width with validation."""
        if not isinstance(value, (int, float)):
            raise TypeError("Width must be a number")
        if value <= 0:
            raise ValueError("Width must be positive")
        self._width = value
    
    # === Height Property ===
    @property
    def height(self):
        """Get height."""
        return self._height
    
    @height.setter
    def height(self, value):
        """Set height with validation."""
        if not isinstance(value, (int, float)):
            raise TypeError("Height must be a number")
        if value <= 0:
            raise ValueError("Height must be positive")
        self._height = value
    
    # === Computed Properties (Read-only) ===
    @property
    def area(self):
        """Computed: area = width * height."""
        return self._width * self._height
    
    @property
    def perimeter(self):
        """Computed: perimeter = 2 * (width + height)."""
        return 2 * (self._width + self._height)
    
    @property
    def diagonal(self):
        """Computed: diagonal = sqrt(width² + height²)."""
        return math.sqrt(self._width ** 2 + self._height ** 2)
    
    def __str__(self):
        return f"Rectangle({self._width} x {self._height})"

# Usage - Clean!
rect = Rectangle(5, 3)

print("Rectangle with @property:")
print("=" * 50)

print(f"\n{rect}")
print(f"Width: {rect.width}")
print(f"Height: {rect.height}")
print(f"Area: {rect.area}")
print(f"Perimeter: {rect.perimeter}")
print(f"Diagonal: {rect.diagonal:.2f}")

# Modify
rect.width = 10
print(f"\nAfter width = 10:")
print(f"Area: {rect.area}")

# Try to set computed property
try:
    rect.area = 100  # Read-only!
except AttributeError as e:
    print(f"\nCan't set area: {e}")

---
<a id='6'></a>
## 6. Computed Properties

### 6.1 Read-Only Computed Properties

Computed properties là properties được **tính toán** từ các attributes khác.

In [ ]:
# Computed Properties Example

from datetime import date, datetime

class Person:
    """Person với computed properties."""
    
    def __init__(self, first_name, last_name, birth_date):
        self.first_name = first_name
        self.last_name = last_name
        self._birth_date = birth_date  # Store as date object
    
    # === Computed Properties ===
    
    @property
    def full_name(self):
        """Computed from first_name and last_name."""
        return f"{self.first_name} {self.last_name}"
    
    @property
    def age(self):
        """Computed from birth_date."""
        today = date.today()
        age = today.year - self._birth_date.year
        # Adjust if birthday hasn't occurred this year
        if (today.month, today.day) < (self._birth_date.month, self._birth_date.day):
            age -= 1
        return age
    
    @property
    def is_adult(self):
        """Computed from age."""
        return self.age >= 18
    
    @property
    def email(self):
        """Computed email from name."""
        return f"{self.first_name.lower()}.{self.last_name.lower()}@example.com"
    
    @property
    def birth_year(self):
        """Computed from birth_date."""
        return self._birth_date.year
    
    @property
    def zodiac_sign(self):
        """Computed zodiac sign from birth_date."""
        month = self._birth_date.month
        day = self._birth_date.day
        
        signs = [
            ((1, 20), "Aquarius"), ((2, 19), "Pisces"), ((3, 21), "Aries"),
            ((4, 20), "Taurus"), ((5, 21), "Gemini"), ((6, 21), "Cancer"),
            ((7, 23), "Leo"), ((8, 23), "Virgo"), ((9, 23), "Libra"),
            ((10, 23), "Scorpio"), ((11, 22), "Sagittarius"), ((12, 22), "Capricorn")
        ]
        
        for (end_month, end_day), sign in signs:
            if month == end_month and day <= end_day:
                return sign
            if month < end_month:
                return sign
        return "Capricorn"

# Test
person = Person("John", "Doe", date(1990, 5, 15))

print("Computed Properties Demo:")
print("=" * 50)

print(f"First name: {person.first_name}")
print(f"Last name: {person.last_name}")
print(f"Full name: {person.full_name}")  # Computed
print(f"Age: {person.age}")  # Computed
print(f"Is adult: {person.is_adult}")  # Computed
print(f"Email: {person.email}")  # Computed
print(f"Birth year: {person.birth_year}")  # Computed
print(f"Zodiac: {person.zodiac_sign}")  # Computed

# Change name -> full_name and email auto update
person.first_name = "Jane"
print(f"\nAfter changing first_name to 'Jane':")
print(f"Full name: {person.full_name}")
print(f"Email: {person.email}")

### 6.2 Computed Property với Setter

In [ ]:
# Computed Property with Setter

class Temperature:
    """Temperature class với multiple scales."""
    
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    # === Celsius (stored value) ===
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    # === Fahrenheit (computed) ===
    @property
    def fahrenheit(self):
        """Convert from Celsius."""
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        """Set via Fahrenheit -> convert to Celsius."""
        self.celsius = (value - 32) * 5/9  # Uses celsius setter for validation
    
    # === Kelvin (computed) ===
    @property
    def kelvin(self):
        """Convert from Celsius."""
        return self._celsius + 273.15
    
    @kelvin.setter
    def kelvin(self, value):
        """Set via Kelvin -> convert to Celsius."""
        self.celsius = value - 273.15
    
    def __str__(self):
        return f"{self._celsius:.2f}°C = {self.fahrenheit:.2f}°F = {self.kelvin:.2f}K"

# Test
temp = Temperature(25)

print("Temperature with Multiple Scales:")
print("=" * 50)

print(f"Initial: {temp}")

# Set via different scales
temp.fahrenheit = 212  # Boiling point
print(f"\nSet fahrenheit = 212:")
print(f"Result: {temp}")

temp.kelvin = 0  # Absolute zero
print(f"\nSet kelvin = 0:")
print(f"Result: {temp}")

temp.celsius = 37  # Body temperature
print(f"\nSet celsius = 37:")
print(f"Result: {temp}")

---
<a id='7'></a>
## 7. Property Deleter

### 7.1 Cú pháp

`@property_name.deleter` cho phép define behavior khi `del obj.property` được gọi.

In [ ]:
# Property Deleter Example

class Person:
    """Person with name property including deleter."""
    
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        """Get name."""
        if self._name is None:
            return "[DELETED]"
        return self._name
    
    @name.setter
    def name(self, value):
        """Set name with validation."""
        if not value or not value.strip():
            raise ValueError("Name cannot be empty")
        self._name = value.strip().title()
    
    @name.deleter
    def name(self):
        """Delete name - set to None."""
        print(f"Deleting name '{self._name}'...")
        self._name = None

# Test
print("Property Deleter Demo:")
print("=" * 50)

person = Person("john doe")
print(f"Name: {person.name}")

person.name = "jane smith"
print(f"After setting: {person.name}")

del person.name  # Calls deleter
print(f"After deleting: {person.name}")

In [ ]:
# Practical Deleter - Cache Invalidation

class DataProcessor:
    """Data processor with cached result."""
    
    def __init__(self, data):
        self._data = data
        self._cached_result = None
    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        self._data = value
        # Invalidate cache when data changes
        del self.result
    
    @property
    def result(self):
        """Get processed result (cached)."""
        if self._cached_result is None:
            print("  [Computing result...]")
            # Expensive computation
            self._cached_result = sum(self._data) * 2
        else:
            print("  [Using cached result]")
        return self._cached_result
    
    @result.deleter
    def result(self):
        """Clear cache."""
        print("  [Cache cleared]")
        self._cached_result = None

# Test
print("Cache Invalidation Demo:")
print("=" * 50)

proc = DataProcessor([1, 2, 3, 4, 5])

print(f"\nFirst access:")
print(f"Result: {proc.result}")

print(f"\nSecond access (cached):")
print(f"Result: {proc.result}")

print(f"\nChange data:")
proc.data = [10, 20, 30]

print(f"\nAccess after data change:")
print(f"Result: {proc.result}")

---
<a id='8'></a>
## 8. Data Validation với Properties

### 8.1 Validation Patterns

In [ ]:
# Comprehensive Data Validation

import re
from datetime import date

class User:
    """
    User class với comprehensive validation.
    """
    
    def __init__(self, username, email, age, password):
        # Initialize private attributes
        self._username = None
        self._email = None
        self._age = None
        self.__password_hash = None
        
        # Set through properties (triggers validation)
        self.username = username
        self.email = email
        self.age = age
        self.password = password
    
    # === Username ===
    @property
    def username(self):
        return self._username
    
    @username.setter
    def username(self, value):
        # Type check
        if not isinstance(value, str):
            raise TypeError("Username must be a string")
        
        # Length check
        if len(value) < 3 or len(value) > 20:
            raise ValueError("Username must be 3-20 characters")
        
        # Format check
        if not re.match(r'^[a-zA-Z][a-zA-Z0-9_]*$', value):
            raise ValueError(
                "Username must start with letter and contain only letters, numbers, underscore"
            )
        
        self._username = value
    
    # === Email ===
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        if not isinstance(value, str):
            raise TypeError("Email must be a string")
        
        # Email regex pattern
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, value):
            raise ValueError("Invalid email format")
        
        self._email = value.lower()
    
    # === Age ===
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise TypeError("Age must be an integer")
        
        if value < 0 or value > 150:
            raise ValueError("Age must be between 0 and 150")
        
        self._age = value
    
    # === Password (write-only) ===
    @property
    def password(self):
        raise AttributeError("Password is write-only")
    
    @password.setter
    def password(self, value):
        if not isinstance(value, str):
            raise TypeError("Password must be a string")
        
        # Password strength validation
        if len(value) < 8:
            raise ValueError("Password must be at least 8 characters")
        if not re.search(r'[A-Z]', value):
            raise ValueError("Password must contain uppercase letter")
        if not re.search(r'[a-z]', value):
            raise ValueError("Password must contain lowercase letter")
        if not re.search(r'[0-9]', value):
            raise ValueError("Password must contain digit")
        
        # Hash and store
        import hashlib
        self.__password_hash = hashlib.sha256(value.encode()).hexdigest()
    
    # === Computed Properties ===
    @property
    def is_adult(self):
        return self._age >= 18
    
    def verify_password(self, password):
        """Verify password without exposing hash."""
        import hashlib
        return hashlib.sha256(password.encode()).hexdigest() == self.__password_hash
    
    def __str__(self):
        return f"User({self._username}, {self._email}, age={self._age})"

# Test
print("Data Validation Demo:")
print("=" * 50)

# Valid user
user = User("john_doe", "john@example.com", 25, "SecurePass123")
print(f"Created: {user}")
print(f"Is adult: {user.is_adult}")
print(f"Password verify: {user.verify_password('SecurePass123')}")

# Test validations
print("\nTesting validations:")

# Invalid username
try:
    user.username = "ab"  # Too short
except ValueError as e:
    print(f"  Username error: {e}")

# Invalid email
try:
    user.email = "invalid-email"
except ValueError as e:
    print(f"  Email error: {e}")

# Invalid age
try:
    user.age = -5
except ValueError as e:
    print(f"  Age error: {e}")

# Weak password
try:
    user.password = "weak"
except ValueError as e:
    print(f"  Password error: {e}")

# Try to read password
try:
    _ = user.password
except AttributeError as e:
    print(f"  Password read error: {e}")

---
<a id='9'></a>
## 9. Read-Only và Write-Only Properties

### 9.1 Read-Only Properties

In [ ]:
# Read-Only Properties

from datetime import datetime

class Order:
    """Order với read-only properties."""
    
    _order_counter = 0
    
    def __init__(self, customer, items):
        Order._order_counter += 1
        self._order_id = f"ORD-{Order._order_counter:05d}"
        self._customer = customer
        self._items = list(items)  # Copy list
        self._created_at = datetime.now()
        self._status = "pending"
    
    # === Read-Only Properties (no setter) ===
    
    @property
    def order_id(self):
        """Order ID - immutable."""
        return self._order_id
    
    @property
    def customer(self):
        """Customer name - immutable."""
        return self._customer
    
    @property
    def created_at(self):
        """Creation timestamp - immutable."""
        return self._created_at
    
    @property
    def items(self):
        """Return copy to prevent modification."""
        return tuple(self._items)  # Return immutable tuple
    
    @property
    def total(self):
        """Computed total."""
        return sum(item['price'] * item['quantity'] for item in self._items)
    
    # === Status can be updated (limited values) ===
    
    VALID_STATUSES = {'pending', 'confirmed', 'shipped', 'delivered', 'cancelled'}
    
    @property
    def status(self):
        return self._status
    
    @status.setter
    def status(self, value):
        if value not in self.VALID_STATUSES:
            raise ValueError(f"Invalid status. Must be one of: {self.VALID_STATUSES}")
        self._status = value
    
    def __str__(self):
        return f"Order({self._order_id}, {self._customer}, ${self.total:.2f}, {self._status})"

# Test
print("Read-Only Properties Demo:")
print("=" * 50)

order = Order("John Doe", [
    {'name': 'Laptop', 'price': 999.99, 'quantity': 1},
    {'name': 'Mouse', 'price': 29.99, 'quantity': 2}
])

print(f"Order: {order}")
print(f"Order ID: {order.order_id}")
print(f"Customer: {order.customer}")
print(f"Created: {order.created_at}")
print(f"Items: {order.items}")
print(f"Total: ${order.total:.2f}")

# Try to modify read-only
print("\nTrying to modify read-only properties:")

try:
    order.order_id = "NEW-ID"
except AttributeError as e:
    print(f"  order_id: {e}")

# Status can be changed (with validation)
order.status = "confirmed"
print(f"\nStatus updated: {order.status}")

try:
    order.status = "invalid_status"
except ValueError as e:
    print(f"Invalid status: {e}")

### 9.2 Write-Only Properties

In [ ]:
# Write-Only Properties (e.g., Password)

import hashlib
import secrets

class SecureAccount:
    """Account với write-only password."""
    
    def __init__(self, username, password):
        self._username = username
        self.__password_hash = None
        self.__salt = None
        self.password = password  # Use setter
    
    @property
    def username(self):
        return self._username
    
    # === Write-Only Password ===
    
    @property
    def password(self):
        """Password cannot be read."""
        raise AttributeError(
            "Password is write-only. Use verify_password() to check."
        )
    
    @password.setter
    def password(self, value):
        """Set password - hashed with salt."""
        if len(value) < 8:
            raise ValueError("Password must be at least 8 characters")
        
        # Generate random salt
        self.__salt = secrets.token_hex(16)
        
        # Hash password with salt
        salted = f"{self.__salt}{value}"
        self.__password_hash = hashlib.sha256(salted.encode()).hexdigest()
    
    def verify_password(self, password):
        """Verify password without exposing hash."""
        salted = f"{self.__salt}{password}"
        input_hash = hashlib.sha256(salted.encode()).hexdigest()
        return secrets.compare_digest(input_hash, self.__password_hash)
    
    def __str__(self):
        return f"SecureAccount({self._username})"

# Test
print("Write-Only Password Demo:")
print("=" * 50)

account = SecureAccount("alice", "MySecurePass123")
print(f"Account: {account}")

# Verify password
print(f"\nVerify correct password: {account.verify_password('MySecurePass123')}")
print(f"Verify wrong password: {account.verify_password('WrongPass')}")

# Cannot read password
print("\nTrying to read password:")
try:
    _ = account.password
except AttributeError as e:
    print(f"  Error: {e}")

# Can change password
account.password = "NewSecurePass456"
print(f"\nAfter password change:")
print(f"Verify old password: {account.verify_password('MySecurePass123')}")
print(f"Verify new password: {account.verify_password('NewSecurePass456')}")

---
<a id='10'></a>
## 10. Ví dụ thực tế

### 10.1 Product Inventory System

In [ ]:
# Product Inventory System

from datetime import datetime
from decimal import Decimal

class Product:
    """
    Product với encapsulated price và inventory.
    """
    
    _id_counter = 0
    
    def __init__(self, name, price, quantity, category="General"):
        Product._id_counter += 1
        self._id = f"PROD-{Product._id_counter:05d}"
        self._name = name
        self._price = Decimal('0')
        self._quantity = 0
        self._category = category
        self._discount_percent = Decimal('0')
        self._created_at = datetime.now()
        
        # Set through properties
        self.price = price
        self.quantity = quantity
    
    # === Read-Only ===
    
    @property
    def id(self):
        return self._id
    
    @property
    def created_at(self):
        return self._created_at
    
    # === Name ===
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not value or not value.strip():
            raise ValueError("Product name cannot be empty")
        self._name = value.strip()
    
    # === Price ===
    
    @property
    def price(self):
        """Base price."""
        return float(self._price)
    
    @price.setter
    def price(self, value):
        if not isinstance(value, (int, float, Decimal)):
            raise TypeError("Price must be a number")
        if value < 0:
            raise ValueError("Price cannot be negative")
        self._price = Decimal(str(value))
    
    # === Quantity ===
    
    @property
    def quantity(self):
        return self._quantity
    
    @quantity.setter
    def quantity(self, value):
        if not isinstance(value, int):
            raise TypeError("Quantity must be an integer")
        if value < 0:
            raise ValueError("Quantity cannot be negative")
        self._quantity = value
    
    # === Discount ===
    
    @property
    def discount_percent(self):
        return float(self._discount_percent)
    
    @discount_percent.setter
    def discount_percent(self, value):
        if not isinstance(value, (int, float, Decimal)):
            raise TypeError("Discount must be a number")
        if not 0 <= value <= 100:
            raise ValueError("Discount must be between 0 and 100")
        self._discount_percent = Decimal(str(value))
    
    # === Computed Properties ===
    
    @property
    def discount_amount(self):
        """Discount amount in currency."""
        return float(self._price * self._discount_percent / 100)
    
    @property
    def final_price(self):
        """Price after discount."""
        return float(self._price - self._price * self._discount_percent / 100)
    
    @property
    def total_value(self):
        """Total inventory value."""
        return self.final_price * self._quantity
    
    @property
    def in_stock(self):
        """Check if in stock."""
        return self._quantity > 0
    
    @property
    def stock_status(self):
        """Stock level status."""
        if self._quantity == 0:
            return "Out of Stock"
        elif self._quantity < 10:
            return "Low Stock"
        elif self._quantity < 50:
            return "In Stock"
        else:
            return "Well Stocked"
    
    # === Methods ===
    
    def add_stock(self, amount):
        """Add to inventory."""
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self._quantity += amount
    
    def remove_stock(self, amount):
        """Remove from inventory."""
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self._quantity:
            raise ValueError("Insufficient stock")
        self._quantity -= amount
    
    def __str__(self):
        if self._discount_percent > 0:
            return (
                f"{self._name} ({self._id})\n"
                f"  Price: ${self.final_price:.2f} (was ${self.price:.2f}, {self.discount_percent:.0f}% off)\n"
                f"  Stock: {self._quantity} ({self.stock_status})"
            )
        return (
            f"{self._name} ({self._id})\n"
            f"  Price: ${self.price:.2f}\n"
            f"  Stock: {self._quantity} ({self.stock_status})"
        )

# Demo
print("Product Inventory System:")
print("=" * 60)

# Create products
laptop = Product("MacBook Pro 14", 1999.99, 25, "Electronics")
mouse = Product("Magic Mouse", 99.99, 5, "Accessories")
cable = Product("USB-C Cable", 19.99, 0, "Accessories")

# Apply discount
mouse.discount_percent = 20

products = [laptop, mouse, cable]

for product in products:
    print(f"\n{product}")

print("\n" + "=" * 60)
print("Inventory Operations:")

# Stock operations
laptop.remove_stock(3)
print(f"\nSold 3 laptops: {laptop.quantity} remaining")

cable.add_stock(100)
print(f"Restocked cables: {cable.stock_status}")

# Total inventory value
total_value = sum(p.total_value for p in products)
print(f"\nTotal inventory value: ${total_value:,.2f}")

### 10.2 Bank Account with Transaction History

In [ ]:
# Bank Account with Full Encapsulation

from datetime import datetime
from decimal import Decimal
import uuid

class BankAccount:
    """
    Bank Account với encapsulation hoàn chỉnh.
    """
    
    MINIMUM_BALANCE = Decimal('10.00')
    
    def __init__(self, owner, initial_balance=0):
        self.__account_number = str(uuid.uuid4())[:8].upper()
        self._owner = owner
        self.__balance = Decimal('0')
        self.__transactions = []
        self.__is_active = True
        self._created_at = datetime.now()
        
        if initial_balance > 0:
            self.deposit(initial_balance, "Initial deposit")
    
    # === Read-Only Properties ===
    
    @property
    def account_number(self):
        """Masked account number."""
        return f"****{self.__account_number[-4:]}"
    
    @property
    def owner(self):
        return self._owner
    
    @property
    def balance(self):
        """Current balance."""
        return float(self.__balance)
    
    @property
    def is_active(self):
        return self.__is_active
    
    @property
    def created_at(self):
        return self._created_at
    
    @property
    def transaction_count(self):
        return len(self.__transactions)
    
    # === Methods ===
    
    def deposit(self, amount, description="Deposit"):
        """Deposit money."""
        self.__check_active()
        
        if not isinstance(amount, (int, float, Decimal)):
            raise TypeError("Amount must be a number")
        
        amount = Decimal(str(amount))
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        
        self.__balance += amount
        self.__record_transaction("DEPOSIT", amount, description)
        
        return float(self.__balance)
    
    def withdraw(self, amount, description="Withdrawal"):
        """Withdraw money."""
        self.__check_active()
        
        if not isinstance(amount, (int, float, Decimal)):
            raise TypeError("Amount must be a number")
        
        amount = Decimal(str(amount))
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        
        if self.__balance - amount < self.MINIMUM_BALANCE:
            raise ValueError(
                f"Insufficient funds. Minimum balance: ${self.MINIMUM_BALANCE}"
            )
        
        self.__balance -= amount
        self.__record_transaction("WITHDRAWAL", amount, description)
        
        return float(self.__balance)
    
    def transfer(self, target_account, amount, description="Transfer"):
        """Transfer to another account."""
        if not isinstance(target_account, BankAccount):
            raise TypeError("Target must be a BankAccount")
        
        self.withdraw(amount, f"Transfer to {target_account.account_number}")
        target_account.deposit(amount, f"Transfer from {self.account_number}")
    
    def get_statement(self, last_n=None):
        """Get transaction statement."""
        transactions = self.__transactions[-last_n:] if last_n else self.__transactions
        
        statement = f"\nAccount Statement for {self.account_number}\n"
        statement += f"Owner: {self._owner}\n"
        statement += "=" * 60 + "\n"
        
        for txn in transactions:
            sign = "+" if txn['type'] == "DEPOSIT" else "-"
            statement += (
                f"{txn['timestamp'].strftime('%Y-%m-%d %H:%M')} | "
                f"{txn['type']:10} | "
                f"{sign}${float(txn['amount']):>10.2f} | "
                f"${float(txn['balance']):>10.2f} | "
                f"{txn['description']}\n"
            )
        
        statement += "=" * 60 + "\n"
        statement += f"Current Balance: ${self.balance:,.2f}\n"
        
        return statement
    
    def close_account(self):
        """Close account."""
        if self.__balance > self.MINIMUM_BALANCE:
            raise ValueError("Withdraw funds before closing")
        self.__is_active = False
        self.__record_transaction("CLOSED", Decimal('0'), "Account closed")
    
    # === Private Methods ===
    
    def __check_active(self):
        """Check if account is active."""
        if not self.__is_active:
            raise ValueError("Account is closed")
    
    def __record_transaction(self, txn_type, amount, description):
        """Record transaction (private)."""
        self.__transactions.append({
            'id': str(uuid.uuid4())[:8],
            'type': txn_type,
            'amount': amount,
            'balance': self.__balance,
            'description': description,
            'timestamp': datetime.now()
        })
    
    def __str__(self):
        status = "Active" if self.__is_active else "Closed"
        return f"Account {self.account_number} ({self._owner}) - ${self.balance:,.2f} [{status}]"

# Demo
print("Bank Account System:")
print("=" * 60)

# Create accounts
alice = BankAccount("Alice Smith", 1000)
bob = BankAccount("Bob Johnson", 500)

print(f"Created: {alice}")
print(f"Created: {bob}")

# Transactions
alice.deposit(250, "Paycheck")
alice.withdraw(100, "ATM")
alice.transfer(bob, 200, "Rent payment")

bob.deposit(150, "Freelance work")

# Statements
print(alice.get_statement())
print(bob.get_statement())

---
<a id='11'></a>
## 11. Best Practices

### 11.1 Dos and Don'ts

| ✅ Do | ❌ Don't |
|-------|----------|
| Use `_name` for protected attributes | Use `__name` everywhere |
| Use `@property` for validation | Use explicit get_/set_ methods |
| Return copies of mutable objects | Return references to internal lists/dicts |
| Validate in setters | Allow any value |
| Document public interface | Expose internal implementation |

### 11.2 When to use what?

```
┌─────────────────────────────────────────────────────────────────┐
│                  WHEN TO USE WHAT?                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   PUBLIC (name)                                                 │
│   ─────────────                                                 │
│   • Part of public API                                          │
│   • Safe to access/modify directly                              │
│   • No validation needed                                        │
│                                                                 │
│   PROTECTED (_name)                                             │
│   ─────────────────                                             │
│   • Internal implementation                                      │
│   • May be used by subclasses                                    │
│   • "Don't touch unless you know what you're doing"             │
│                                                                 │
│   PRIVATE (__name)                                              │
│   ────────────────                                              │
│   • Strictly internal                                            │
│   • Avoid name collision in inheritance                          │
│   • Sensitive data (passwords, keys)                             │
│                                                                 │
│   @PROPERTY                                                     │
│   ─────────                                                     │
│   • Need validation on set                                       │
│   • Computed values                                              │
│   • Read-only or write-only access                               │
│   • Backward compatibility                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Best Practice: Return copies of mutable objects

class ShoppingCart:
    """Shopping cart với proper encapsulation."""
    
    def __init__(self):
        self._items = []  # Internal list
    
    @property
    def items(self):
        """Return a COPY to prevent external modification."""
        return list(self._items)  # ✅ Return copy
        # return self._items  # ❌ Don't do this!
    
    def add_item(self, item):
        """Controlled way to add items."""
        self._items.append(item)
    
    def remove_item(self, item):
        """Controlled way to remove items."""
        self._items.remove(item)

# Test
cart = ShoppingCart()
cart.add_item("Apple")
cart.add_item("Banana")

# Get items
items = cart.items
print(f"Items: {items}")

# Try to modify the returned list
items.append("Hacked Item")  # This modifies the copy, not the original!

print(f"After 'hack': {cart.items}")  # Original unchanged!

---
<a id='12'></a>
## 12. Common Mistakes

### 12.1 Infinite Recursion in Property

In [ ]:
# ❌ MISTAKE: Infinite recursion

class BadExample:
    def __init__(self, value):
        self.value = value  # Calls setter
    
    @property
    def value(self):
        return self.value  # ❌ Calls itself! Infinite recursion!
    
    @value.setter
    def value(self, val):
        self.value = val  # ❌ Calls itself! Infinite recursion!

# This will cause RecursionError
# bad = BadExample(10)

print("❌ BadExample causes RecursionError!")

In [ ]:
# ✅ CORRECT: Use different internal name

class GoodExample:
    def __init__(self, value):
        self.value = value  # Calls setter
    
    @property
    def value(self):
        return self._value  # ✅ Different name!
    
    @value.setter
    def value(self, val):
        self._value = val  # ✅ Different name!

good = GoodExample(10)
print(f"✅ GoodExample works: {good.value}")

### 12.2 Forgetting to call setter in `__init__`

In [ ]:
# ❌ MISTAKE: Bypass validation

class BadAge:
    def __init__(self, age):
        self._age = age  # ❌ Bypasses validation!
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if value < 0:
            raise ValueError("Age cannot be negative")
        self._age = value

# Invalid age is accepted!
bad = BadAge(-10)
print(f"❌ BadAge allows invalid: {bad.age}")

In [ ]:
# ✅ CORRECT: Use property in __init__

class GoodAge:
    def __init__(self, age):
        self.age = age  # ✅ Uses setter!
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if value < 0:
            raise ValueError("Age cannot be negative")
        self._age = value

# Invalid age is rejected!
try:
    good = GoodAge(-10)
except ValueError as e:
    print(f"✅ GoodAge rejects invalid: {e}")

# Valid age works
good = GoodAge(25)
print(f"✅ GoodAge with valid: {good.age}")

---
<a id='13'></a>
## 13. Bài tập thực hành

### Bài 1: Timer Class (Easy)

Tạo `Timer` class với:
- Private `__seconds` để lưu thời gian
- Properties: `hours`, `minutes`, `seconds` (có thể get và set)
- Property `total_seconds` (read-only)
- Validation: không cho phép giá trị âm

In [ ]:
# Bài 1: Timer Class

class Timer:
    """Timer class với encapsulated time."""
    
    def __init__(self, hours=0, minutes=0, seconds=0):
        pass  # TODO: Implement
    
    # TODO: Implement properties

# Test code
# timer = Timer(1, 30, 45)
# print(f"Total seconds: {timer.total_seconds}")  # 5445
# timer.hours = 2
# print(f"After hours=2: {timer.total_seconds}")  # 9045

### Bài 2: Employee Salary System (Medium)

Tạo `Employee` class với:
- Private `__base_salary` (validated: >= 0)
- Protected `_bonus_percent` (validated: 0-100)
- Computed properties: `bonus_amount`, `total_salary`
- Method `give_raise(percent)` để tăng lương
- Salary history (private)

In [ ]:
# Bài 2: Employee Salary System

class Employee:
    """Employee với salary encapsulation."""
    
    def __init__(self, name, base_salary, bonus_percent=10):
        pass  # TODO: Implement
    
    # TODO: Implement properties and methods

# Test code
# emp = Employee("Alice", 50000, 15)
# print(f"Total salary: ${emp.total_salary:,.2f}")
# emp.give_raise(10)  # 10% raise
# print(f"After raise: ${emp.total_salary:,.2f}")

### Bài 3: Secure Config System (Hard)

Tạo `SecureConfig` class để quản lý configuration:
- Encrypted storage cho sensitive values
- Read-only config keys
- Validation cho specific config types
- Audit log cho changes

In [ ]:
# Bài 3: Secure Config System

class SecureConfig:
    """
    Secure configuration manager.
    
    Features:
    - Encrypted sensitive values
    - Validation
    - Audit logging
    """
    
    # Config schema with validation
    SCHEMA = {
        'database_host': {'type': str, 'required': True},
        'database_port': {'type': int, 'min': 1, 'max': 65535},
        'database_password': {'type': str, 'sensitive': True},
        'debug_mode': {'type': bool, 'default': False},
        'max_connections': {'type': int, 'min': 1, 'max': 1000, 'default': 100}
    }
    
    def __init__(self):
        pass  # TODO: Implement
    
    # TODO: Implement get/set with validation, encryption, logging

# Test code
# config = SecureConfig()
# config.set('database_host', 'localhost')
# config.set('database_password', 'secret123')
# print(config.get('database_host'))
# print(config.get('database_password'))  # Should be masked or encrypted
# print(config.get_audit_log())

---

## Tổng kết

### Kiến thức đã học

| Khái niệm | Mô tả | Cú pháp |
|-----------|-------|--------|
| **Public** | Accessible từ mọi nơi | `self.name` |
| **Protected** | Convention - internal use | `self._name` |
| **Private** | Name mangling | `self.__name` → `self._ClassName__name` |
| **@property** | Pythonic getter | `@property` |
| **@x.setter** | Pythonic setter | `@property_name.setter` |
| **@x.deleter** | Pythonic deleter | `@property_name.deleter` |
| **Computed** | Calculated from other attrs | Property without backing attr |
| **Read-only** | Getter without setter | Only `@property` |
| **Write-only** | Setter that raises on get | Getter raises `AttributeError` |

### Best Practices Summary

1. ✅ Use `@property` instead of explicit getters/setters
2. ✅ Validate in setters, not just in `__init__`
3. ✅ Return copies of mutable objects
4. ✅ Use `_name` for protected, `__name` only when needed
5. ✅ Document public interface clearly
6. ✅ Use computed properties for derived values

---

### Tiếp theo: 06 - Abstraction (Trừu tượng hóa)